In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.functions import col, when, to_date, lit, expr, countDistinct, max, min, sum, concat, count, avg, desc, rank
from pyspark.sql.window import Window
import sys

sc.install_pypi_package("pandas")
sc.install_pypi_package("np")

import np
import pandas as pd




VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1604217164490_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/bf/4c/cb7da76f3a5e077e545f9cf8575b8f488a4e8ad60490838f89c5cdd5bb57/pandas-1.1.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl


<br><br><br>

# DATA LOAD

In [2]:
in_bucket = "s3://tfmbigdata/Files_Segunda_Entrega/"

tickets = spark.read.format("json").load(in_bucket+'tickets*.json')
customers = spark.read.format("json").load(in_bucket+'customers.json')
products = spark.read.format("json").load(in_bucket+'products.json')
stores = spark.read.format("json").load(in_bucket+'stores.json')
clasificacion = spark.read.option("header", True).format("csv").load(in_bucket+'clasificacion_productos.csv')
prediction = spark.read.format("json").load('s3://mbde-tfm-grupo3/prediction/*.json')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br><br>

# DISTRIBUCIÓN DEL % DE GASTO POR SEGMENTO

In [3]:
prediction = prediction.crossJoin(prediction.groupby().agg(f.sum('monetary').alias('total_spend')))
 
prediction.head()    
    
aux = prediction.groupBy("segmento")\
                         .agg(sum("monetary").alias("spend_by_segment"),
                            max("total_spend").alias("total_spend"))
 
spend = aux.withColumn("Spend %", expr("spend_by_segment/total_spend")).orderBy("segmento").toPandas()
spend.head(10)
               



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    segmento  spend_by_segment  total_spend   Spend %
0       best        8937773.96  22455132.54  0.398028
1       lost        1089857.15  22455132.54  0.048535
2      loyal        1672148.96  22455132.54  0.074466
3        new        5474910.14  22455132.54  0.243816
4  promising        2494364.03  22455132.54  0.111082
5       risk        2786078.30  22455132.54  0.124073

<br><br><br>

# DISTRIBUCIÓN DE LOS CLIENTES POR SEGMENTO

In [4]:
total_customer_count = prediction.count()
prediction = prediction.withColumn('total_customer_count',lit(total_customer_count))


customer_frequency = prediction.groupBy("segmento")\
                        .agg(count("customerid").alias("customer_count"),\
                             max("total_customer_count").alias("total_customer_count"))
 
customer_frequency = customer_frequency.withColumn("Customer%", expr("customer_count/total_customer_count")).orderBy("segmento").toPandas()

customer_frequency.head(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    segmento  customer_count  total_customer_count  Customer%
0       best           12123                 49729   0.243781
1       lost            6689                 49729   0.134509
2      loyal            6283                 49729   0.126345
3        new            8035                 49729   0.161576
4  promising            8782                 49729   0.176597
5       risk            7817                 49729   0.157192

<br><br><br>

# PROMEDIO RMF POR SEGMENTO

In [5]:
rfm_avg_by_segment = prediction.groupBy("segmento")\
                        .agg(avg("frequency").alias("frequency_avg"),\
                             avg("monetary").alias("monetary_avg"),\
                             avg("recency").alias("recency_avg"))

rfm_avg_by_segment = rfm_avg_by_segment.orderBy("segmento").toPandas()
rfm_avg_by_segment.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    segmento  frequency_avg  monetary_avg  recency_avg
0       best      28.989524    737.257606     1.730925
1       lost      11.547615    162.932748    19.086560
2      loyal      24.337259    266.138622     3.406016
3        new      17.484256    681.382718     9.787181
4  promising      11.751993    284.031431     1.396607
5       risk      12.381476    356.412729    16.614942

<br><br><br>

# GLOBAL DATASET GENERATION

In [6]:

prediction_tickets = prediction.alias("p")\
                     .join(tickets.alias("t"),prediction.customerid == tickets.customerid,how='left')\
                     .select(
                       
                        col("p.segmento").alias("segmento"),
                        col("p.prediction").alias("prediction"),
                        col("t.ticketid").alias("ticketid"),
                        col("t.productid").alias("productid"),
                        col("t.customerid").alias("customerid"),
                        col("t.storeid").alias("storeid"),
                        col("t.originalamount").alias("originalamount"),
                        col("t.extendedamount").alias("extendedamount"),
                        col("t.quantity").alias("quantity"),
                        col("t.datekey").alias("datekey"),
                        col("t.cardtype").alias("cardtype"),
                        col("t.totaldiscount").alias("totaldiscount")
                     )


prediction_tickets_customer = prediction_tickets.alias("pt")\
                     .join(customers.alias("c"),prediction_tickets.customerid == customers.customerid,how='left')\
                     .select(
                        "pt.*",
                         col("c.age").alias("age"),
                         col("c.gender").alias("gender") 
                     )

prediction_tickets_customer_store = prediction_tickets_customer.alias("ptc")\
                     .join(stores.alias("s"),prediction_tickets_customer.storeid == stores.storeid,how='left')\
                     .select(
                        "ptc.*",
                         col("s.city").alias("city")
                     )

prediction_tickets_customer_store_products = prediction_tickets_customer_store.alias("ptcs")\
                     .join(products.alias("p"),prediction_tickets_customer_store.productid == products.productid,how='left')\
                     .select(
                        "ptcs.*",
                         col("p.merchandisegroup").alias("merchandisegroup"),
                         col("p.merchandisefamily").alias("merchandisefamily"),             
                         col("p.vatdescription").alias("vatdescription"),
                         col("p.article").alias("article"),
                         
                        
                     )


prediction_with_clasification = prediction_tickets_customer_store_products.alias("ptcs")\
                    .join(clasificacion.alias("c"),prediction_tickets_customer_store_products.merchandisefamily == clasificacion.merchandisefamily,how='left')\
                     .select(
                        "ptcs.*",
                         col("c.healthy").alias("healthy"),
                         col("c.unhealthy").alias("unhealthy")
                    )

    


dataset = prediction_with_clasification.drop_duplicates()





VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br><br>

<br><br><br>

# MEDIA DE EDAD POR SEGMENTO

In [7]:
df = dataset.select("segmento","customerid","age").filter(dataset.age.isNotNull()).drop_duplicates()

age_avg_by_segment = df.groupBy("segmento")\
                        .agg(avg("age").alias("age_avg")).toPandas()

age_avg_by_segment.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    segmento    age_avg
0        new  46.191441
1       lost  45.279518
2  promising  44.089908
3      loyal  47.042017
4       best  47.091270
5       risk  44.540254

<br><br><br>

# %GENERO POR SEGMENTO

In [8]:
dataset = dataset.withColumn("new_gender", when(col("gender") == "M","Male")\
                                 .when(col("gender") == "F","Female")\
                                 .otherwise("Unknown"))

df = dataset.select("new_gender","segmento","customerid").filter(dataset.new_gender != "Unknown").drop_duplicates()


total_gender_count_by_segment = df.groupBy("segmento")\
                        .agg(count("customerid").alias("total_gender_count"))


gender_count_by_segment = df.groupBy("segmento","new_gender")\
                        .agg(count("customerid").alias("gender_count"))


gender_analysis = gender_count_by_segment.alias("gend")\
                     .join(total_gender_count_by_segment.alias("totgend"), gender_count_by_segment.segmento == total_gender_count_by_segment.segmento,how='left')\
                     .select(         
                        col("gend.segmento").alias("segmento"),
                        col("gend.new_gender").alias("new_gender"),
                        col("gend.gender_count").alias("gender_count"),
                        col("totgend.total_gender_count").alias("total_gender_count")
                     )

gender_freq = gender_analysis.withColumn("Gendegender_countr%", expr("gender_count/total_gender_count")).orderBy("segmento").toPandas()

gender_freq.head(12)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     segmento new_gender  gender_count  total_gender_count  Gendegender_countr%
0        best       Male           344                1005             0.342289
1        best     Female           661                1005             0.657711
2        lost       Male           143                 415             0.344578
3        lost     Female           272                 415             0.655422
4       loyal       Male           182                 474             0.383966
5       loyal     Female           292                 474             0.616034
6         new       Male           157                 444             0.353604
7         new     Female           287                 444             0.646396
8   promising     Female           367                 541             0.678373
9   promising       Male           174                 541             0.321627
10       risk       Male           172                 471             0.365180
11       risk     Female           299  

<br><br><br>

# TOP 10 MERCHANDISE FAMILY BY SEGMENT

In [9]:
df = dataset.select("segmento","productid","merchandisefamily").filter(dataset.merchandisefamily.isNotNull())

merchandisefamily = df.groupBy("segmento","merchandisefamily")\
                        .agg(count("productid").alias("product_count"))


window = Window.partitionBy(merchandisefamily['segmento']).orderBy(merchandisefamily['product_count'].desc())

merchandisefamily.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10).show(500) 



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+----+
| segmento|   merchandisefamily|product_count|rank|
+---------+--------------------+-------------+----+
|      new|Baguette, panecil...|        77025|   1|
|      new|               Pollo|        53148|   2|
|      new|               Cerdo|        51635|   3|
|      new|     Frutas de hueso|        38189|   4|
|      new|             Tomates|        34212|   5|
|      new|Verduras (Feingem...|        32936|   6|
|      new|    Bananas/Plátanos|        31577|   7|
|      new|  Pasteles, bollería|        27884|   8|
|      new|    Patatas/cebollas|        27397|   9|
|      new|              Huevos|        27369|  10|
|     lost|Baguette, panecil...|        31532|   1|
|     lost|  Pasteles, bollería|        12199|   2|
|     lost|               Pollo|         9016|   3|
|     lost|     Frutas de hueso|         8852|   4|
|     lost|               Cerdo|         8557|   5|
|     lost|    Bananas/Plátanos|         8182|   6|
|     lost| 

<br><br><br>

# LAST 10 MERCHANDISE FAMILY BY SEGMENT

In [10]:
window = Window.partitionBy(merchandisefamily['segmento']).orderBy(merchandisefamily['product_count'].asc())

merchandisefamily.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10).show(500) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+----+
| segmento|   merchandisefamily|product_count|rank|
+---------+--------------------+-------------+----+
|      new|  Conservas de fruta|            2|   1|
|      new|Medicamentos de v...|           16|   2|
|      new|              Bolsas|           16|   2|
|      new|Hot Convenience F...|           32|   4|
|      new|Vino rosado de Eu...|           35|   5|
|      new|Especias, mezcla ...|           64|   6|
|      new|       Otros aceites|           70|   7|
|      new|              Quesos|           85|   8|
|      new|Bebidas espirituo...|           90|   9|
|      new|            Bombones|          109|  10|
|     lost|  Conservas de fruta|            1|   1|
|     lost|              Bolsas|            4|   2|
|     lost|Medicamentos de v...|            5|   3|
|     lost|              Quesos|            7|   4|
|     lost|Vino rosado de Eu...|            9|   5|
|     lost|Especias, mezcla ...|           11|   6|
|     lost| 

<br><br><br>

# Promedio de cantidad de productos por compra

In [11]:
df = dataset.select("segmento","customerid","ticketid","productid","quantity").drop_duplicates()

average_quantity_per_customer =  df.groupBy("segmento", "customerid")\
                        .agg((f.count("productid")/f.countDistinct("ticketid")).alias("avg_purchase_count")).sort(desc("avg_purchase_count"))

average_quantity_per_segment = average_quantity_per_customer.groupBy("segmento")\
                        .agg(f.mean("avg_purchase_count").alias("avg_purchase_count"))


average_quantity_per_segment.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+
| segmento|avg_purchase_count|
+---------+------------------+
|      new|19.136314667395883|
|     lost|  7.84063677637989|
|promising|11.951424974510541|
|    loyal|6.6475823392820415|
|     best|12.874095930968231|
|     risk|14.364542478402068|
+---------+------------------+

<br><br><br>

<br><br><br>

# Distribucion de productos de acuerdo a su VAT

In [12]:
df = dataset.select("segmento","customerid","ticketid","productid","vatdescription").filter(dataset.vatdescription.isNotNull()).drop_duplicates()

vat_distribution =  df.groupBy("segmento", "vatdescription")\
                        .agg(f.count("productid").alias("vat_quantity")).sort(desc("segmento"))\
                        .withColumn('Relative_Freq', f.col('vat_quantity')/f.sum('vat_quantity').over(Window.partitionBy('segmento'))*100)\
                        .drop("vat_quantity")


vat_distribution.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+------------------+
| segmento|vatdescription|     Relative_Freq|
+---------+--------------+------------------+
|     risk|     Ordinario|  5.45607935806331|
|     risk|      Reducido| 47.10461221991772|
|     risk| Superreducido|47.439308422018975|
|promising| Superreducido| 46.96376546645645|
|promising|     Ordinario| 5.193562134807649|
|promising|      Reducido|  47.8426723987359|
|      new|     Ordinario| 5.422638078331552|
|      new| Superreducido|47.297910110362395|
|      new|      Reducido| 47.27945181130605|
|    loyal| Superreducido|50.359052297337634|
|    loyal|      Reducido| 46.08326322360029|
|    loyal|     Ordinario|3.5576844790620674|
|     lost|      Reducido|47.367764759129344|
|     lost| Superreducido| 48.26465059784692|
|     lost|     Ordinario| 4.367584643023734|
|     best| Superreducido| 48.37938330987595|
|     best|      Reducido| 46.76233505625592|
|     best|     Ordinario| 4.858281633868127|
+---------+--------------+--------

<br><br><br>

<br><br><br>

# Healthy vs Unhealthy Products

In [13]:
df = dataset.select("segmento","customerid","ticketid","productid","healthy","unhealthy").filter(dataset.healthy.isNotNull()).drop_duplicates()

clasification_distribution =  df.groupBy("segmento", "healthy", "unhealthy")\
                        .agg(f.count("productid").alias("product_quantity")).sort(desc("segmento"))\
                        .withColumn('Relative_Freq', f.col('product_quantity')/f.sum('product_quantity').over(Window.partitionBy('segmento'))*100)\
                        .drop("product_quantity")
                       

clasification_distribution.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+---------+------------------+
| segmento|healthy|unhealthy|     Relative_Freq|
+---------+-------+---------+------------------+
|     risk|      0|        0| 5.668712599130977|
|     risk|      0|        1| 32.47910673279667|
|     risk|      1|        0| 61.85218066807236|
|promising|      1|        0| 59.92817764809853|
|promising|      0|        0|5.8556726097792495|
|promising|      0|        1| 34.21614974212222|
|      new|      1|        0| 64.08576376845934|
|      new|      0|        1|30.413148216192827|
|      new|      0|        0| 5.501088015347827|
|    loyal|      0|        1| 40.43310792063587|
|    loyal|      0|        0| 5.705588892938943|
|    loyal|      1|        0| 53.86130318642519|
|     lost|      0|        0|  6.11913131678037|
|     lost|      1|        0|56.134183675832396|
|     lost|      0|        1|37.746685007387235|
|     best|      1|        0|61.200201122179145|
|     best|      0|        0| 5.381051011839487|
|     best|      0| 

<br><br><br>

<br><br><br>

# Distribucion de Productos comprados con Descuento

In [14]:
df = dataset.select("segmento","customerid","ticketid","productid","originalamount","extendedamount").filter(dataset.originalamount.isNotNull()).drop_duplicates()

discount_distribution =  df.groupBy("segmento", "ticketid", "productid")\
                        .agg(when(f.sum('originalamount')-f.sum('extendedamount')>0,"YES").otherwise("NO").alias("Discount"))\
                        .sort(desc("segmento"))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
discount_distribution = discount_distribution.groupBy("segmento","Discount")\
                        .agg(f.count('productid').alias("Total_Products"))\
                        .withColumn('Percentage', f.col('Total_Products')/f.sum('Total_Products').over(Window.partitionBy('segmento'))*100)\
                        .drop("Total_Products")
                       

discount_distribution.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+------------------+
| segmento|Discount|        Percentage|
+---------+--------+------------------+
|     risk|      NO| 83.00535241474061|
|     risk|     YES|16.994647585259397|
|promising|      NO| 82.37159854669945|
|promising|     YES| 17.62840145330055|
|      new|      NO| 81.54887061259244|
|      new|     YES|18.451129387407562|
|    loyal|      NO| 74.81516004897856|
|    loyal|     YES| 25.18483995102144|
|     lost|      NO|  81.6323418560524|
|     lost|     YES|18.367658143947597|
|     best|      NO| 76.57293386296733|
|     best|     YES|23.427066137032668|
+---------+--------+------------------+

<br><br><br>

<br><br><br>

# Total Gastos

In [15]:
total_amount = dataset.agg((f.sum("extendedamount")/1000000).alias("Total_in_Millions"))
total_amount.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
| Total_in_Millions|
+------------------+
|19.492447149998647|
+------------------+

<br><br><br>

# Top Products

In [16]:
df = dataset.select("segmento","article").filter(dataset.article.isNotNull())

bestproducts = df.groupBy("segmento","article")\
                        .agg(count("article").alias("product_count"))                        


window = Window.partitionBy(bestproducts['segmento']).orderBy(bestproducts['product_count'].desc())

bestproducts.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 5).show(30) 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+-------------+----+
| segmento|           article|product_count|rank|
+---------+------------------+-------------+----+
|      new|            Banana|        16740|   1|
|      new|   Huevos L libres|        12259|   2|
|      new|Plátano Canario FP|        10266|   3|
|      new|      Barra de pan|        10135|   4|
|      new|Melón Piel de Sapo|         9591|   5|
|     lost|            Banana|         4493|   1|
|     lost|      Barra de pan|         3967|   2|
|     lost|          Baguette|         3545|   3|
|     lost|Plátano Canario FP|         2784|   4|
|     lost|Melón Piel de Sapo|         2763|   5|
|promising|            Banana|         8270|   1|
|promising|      Barra de pan|         5778|   2|
|promising|          Baguette|         5684|   3|
|promising|   Huevos L libres|         5446|   4|
|promising|Melón Piel de Sapo|         4879|   5|
|    loyal|      Barra de pan|         9976|   1|
|    loyal|            Banana|         9186|   2|


<br><br><br>

# Top Cities

In [17]:
df = dataset.select("segmento","city","customerid").filter(dataset.city.isNotNull())

top_cities = df.groupBy("city","segmento")\
                        .agg(countDistinct("customerid").alias("customer_count"))\
                        .withColumn('Total_city', f.sum('customer_count').over(Window.partitionBy('city')))\
                        .sort(desc("Total_city"))

    
top_cities.show(60)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------+--------------+----------+
|             city| segmento|customer_count|Total_city|
+-----------------+---------+--------------+----------+
|           Madrid|    loyal|           619|      2649|
|           Madrid|     lost|           435|      2649|
|           Madrid|     risk|           305|      2649|
|           Madrid|     best|           609|      2649|
|           Madrid|      new|           276|      2649|
|           Madrid|promising|           405|      2649|
|          Sevilla|     lost|           199|      1211|
|          Sevilla|      new|           162|      1211|
|          Sevilla|     best|           292|      1211|
|          Sevilla|    loyal|           194|      1211|
|          Sevilla|promising|           203|      1211|
|          Sevilla|     risk|           161|      1211|
|         Zaragoza|     risk|           163|      1176|
|         Zaragoza|promising|           185|      1176|
|         Zaragoza|     best|           281|    

<br><br><br>

# Distribucion de Genero y Edad

In [18]:
age_range = dataset.select("segmento","customerid","gender","age").filter((dataset.gender == "M") | (dataset.gender == "F") & (dataset.age.isNotNull())).drop_duplicates()

age_range = age_range.withColumn('Age_Range',\
                           when(col("age")<18,"Menor de 18")\
                          .when((col("age")>=18) & (col("age")<35) ,"Entre 18-34")\
                          .when((col("age")>=35) & (col("age")<50) ,"Entre 35-49")\
                          .when((col("age")>=50) & (col("age")<65) ,"Entre 50-64")\
                          .otherwise("Mayor de 65"))
    


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
age_range_dist = age_range.groupBy("segmento","Age_Range","gender")\
                        .agg(countDistinct("customerid").alias("customer_count")).sort(desc("segmento"))

    
age_range_dist.show(72)




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------+------+--------------+
| segmento|  Age_Range|gender|customer_count|
+---------+-----------+------+--------------+
|     risk|Entre 18-34|     F|            56|
|     risk|Entre 35-49|     M|            90|
|     risk|Mayor de 65|     F|            16|
|     risk|Entre 50-64|     M|            47|
|     risk|Entre 18-34|     M|            26|
|     risk|Entre 50-64|     F|            80|
|     risk|Entre 35-49|     F|           147|
|     risk|Mayor de 65|     M|             9|
|promising|Entre 50-64|     F|            87|
|promising|Entre 50-64|     M|            49|
|promising|Entre 35-49|     F|           195|
|promising|Entre 18-34|     M|            33|
|promising|Entre 35-49|     M|            84|
|promising|Mayor de 65|     M|             8|
|promising|Mayor de 65|     F|            17|
|promising|Entre 18-34|     F|            68|
|      new|Mayor de 65|     F|             8|
|      new|Entre 18-34|     F|            40|
|      new|Entre 35-49|     F|    

<br><br><br>